### Estruturação dos módulos de PyTorch

```python
import torch
```
* [`torch.Tensor`](https://pytorch.org/docs/stable/tensors.html) - Tensores (arrays N-D)
* [`torch.nn`](https://pytorch.org/docs/stable/nn.html) - Redes Neurais (_**N**eural **N**etworks_)
* [`torch.optim`](https://pytorch.org/docs/stable/optim.html) - Otimização (_**Optim**ization_)
* [`torch.data`](https://pytorch.org/docs/stable/data.html) - *Datasets* e Ferramentas de Streaming de Dados
* [`torch.autograd`](https://pytorch.org/docs/stable/autograd.html) - Diferenciação Automática (_**Auto**matic Differentiation_)
* [`torch.vision`](https://pytorch.org/docs/stable/torchvision/index.html) - Ferramentas de Manipulação de Imagens e Visão Computacional
* [`torch.audio`](https://pytorch.org/audio/stable/index.html) - Ferramentas de Manipulação de Áudio
* [`torch.jit`](https://pytorch.org/docs/stable/jit.html) - Compilação _**j**ust-**i**n-**t**ime_ de modelos PyTorch em binários


In [6]:
import torchvision
from torchvision import transforms

# MNIST dataset
root_path = 'data'

# Pequena transformação para tensores e normalizando o tamanho
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Train/Test Datasets
full_train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=root_path, train=False, transform=trans)

In [8]:
full_train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### 2. Divisão em Treino e Validação

Usamos `random_split` para dividir o `full_train_dataset` em um conjunto de treino e um de validação. Uma divisão comum é 80% para treino e 20% para validação.

In [9]:
from torch.utils.data import random_split
# Definir os tamanhos para treino e validação
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size

# Dividir o dataset
train_dataset, val_dataset = random_split(
    full_train_dataset, [train_size, val_size])

print(f"Tamanho do dataset de treino: {len(train_dataset)}")
print(f"Tamanho do dataset de validação: {len(val_dataset)}")
print(f"Tamanho do dataset de teste: {len(test_dataset)}")

Tamanho do dataset de treino: 48000
Tamanho do dataset de validação: 12000
Tamanho do dataset de teste: 10000


### 3. Criar DataLoaders

DataLoaders são responsáveis por carregar os dados em lotes (batches), o que é essencial para o treinamento de redes neurais.

In [11]:
from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size, shuffle=False)


### 4. Definir o Modelo (CNN)

Aqui definimos uma arquitetura de Rede Neural Convolucional (CNN).

In [12]:
import torch.nn as nn

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(7 * 7 * 64, 1000),
            nn.ReLU())
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


# Instancia o Model()
model = ConvNet()

print(model)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=3136, out_features=1000, bias=True)
    (1): ReLU()
  )
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)


Quantidade de parametros

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


count_parameters(model)

3199106

In [16]:
from torch.optim import Adam

# Hiperparâmetros
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
epochs = 6

# Instânciar o Otimizador Adam
optimizer = Adam(model.parameters(), lr=learning_rate)

In [17]:
# Isto tem que retornar True
import torch
is_cuda_avaliable = torch.cuda.is_available()
print(is_cuda_avaliable)

False


In [18]:
# Sua GPU
torch.cuda.get_device_name()

AssertionError: Torch not compiled with CUDA enabled

### Enviar Modelo para GPU (se disponível) EX:1

In [23]:
from torch import device


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando o dispositivo: {device}')

Usando o dispositivo: cpu


In [ ]:
# Treinar o Modelo
total_step = len(train_loader)  # quantos batches eu tenho

# Listas vazias
train_loss_list = []
train_acc_list = []
valid_acc_list = []
valid_loss_list = []

best_val_acc = 0

model.to(device) # mover o modelo para GPU EX: 1


for epoch in range(epochs):
    model.train()  # modo de treino
    
    train_loss = 0
    correct =0
    total = 0


    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)  # mover os dados para GPU EX: 1
        labels = labels.to(device)  # mover os dados para GPU EX: 1
                
        # Gera a propagação (feed forward)
        outputs = model(images)

        # Calcula a função-custo
        loss = loss_fn(outputs, labels)
        

        # Retro-propagação (Backprop) e a otimização com Adam
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Acurácia
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        total += labels.size(0)

        if (i + 1) % 100 == 0:
            print(
                f"Época [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], Custo: {round(loss.item(), 3)}, Acurácia: {round((correct / total) * 100, 3)}")
            
    #Media de treino
    train_loss_list.append(train_loss/len(train_loader))
    train_acc_list.append((correct / total) * 100)

#Ex 2 Validação do Modelo
    model.eval()  # modo de avaliação
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)  # mover os dados para GPU EX: 1
            labels = labels.to(device)  # mover os dados para GPU EX: 1

            # Gera a propagação (feed forward)
            outputs = model(images)

            # Calcula a função-custo
            loss = loss_fn(outputs, labels)

           

            # Acurácia
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Média de validação
    val_loss /= len(val_loader)
    val_acc = 100 * correct / total
    valid_loss_list.append(val_loss)
    valid_acc_list.append(val_acc)

    print(f'Validação - Época [{epoch+1}/{epochs}], Custo: {round(val_loss, 3)}, Acurácia: {round(val_acc, 3)}')

    


In [ ]:
total_step = len(train_loader)

train_loss_list = []
train_acc_list = []
valid_loss_list = []
valid_acc_list = []

best_val_acc = 0

model.to(device)

for epoch in range(epochs):
    # ---- Treino ----
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Acumula loss e acurácia
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i + 1) % 100 == 0:
            print(f"Época [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], "
                  f"Custo: {loss.item():.3f}, "
                  f"Acurácia: {(correct/total)*100:.2f}%")

    # Médias de treino
    train_loss /= len(train_loader.dataset)
    train_acc = 100 * correct / total
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    # ---- Validação ----
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = 100 * correct / total
    valid_loss_list.append(val_loss)
    valid_acc_list.append(val_acc)

    print(f"Validação - Época [{epoch+1}/{epochs}], "
          f"Custo: {val_loss:.3f}, Acurácia: {val_acc:.2f}%")

In [ ]:
# 3 - O treinamento imprime o custo e a acurácia no terminal, mas os resultados são mais fáceis de interpretar visualmente. Então, armazene o histórico de perda e acurácia de treino e validação a cada época em listas. Ao final do treinamento, utilize a biblioteca matplotlib para plotar gráficos da perda e da acurácia ao longo das épocas. Isso oferece uma visão clara sobre a convergência do modelo e o surgimento de overfitting.

import matplotlib.pyplot as plt
# Plotando a perda de treino e validação
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss_list, label='Perda de Treino')
plt.plot(valid_loss_list, label='Perda de Validação')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.title('Perda de Treino e Validação ao Longo das Épocas')
plt.grid()
plt.subplot(1, 2, 2)
plt.plot(train_acc_list, label='Acurácia de Treino')
plt.plot(valid_acc_list, label='Acurácia de Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia (%)')
plt.legend()
plt.title('Acurácia de Treino e Validação ao Longo das Épocas')
plt.grid()
plt.show()

In [ ]:
# 4 - Um ponto importante em aprendizagem profunda envolve salvar o modelo treinado para uso posterior. Demonstre como salvar o estado do modelo (model.state_dict()) com torch.save(). Salve o modelo que apresentou o melhor desempenho no conjunto de validação. Mostre também como carregar esses pesos de volta em uma instância do modelo. 
    
# Salva o melhor modelo
from tabnanny import check


checkpoint={
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epochs,
    'train_loss_list': train_loss_list,
    'train_acc_list': train_acc_list,
    'valid_loss_list': valid_loss_list,
    'valid_acc_list': valid_acc_list
}
torch.save(checkpoint, 'best_model.pth')
# Carrega o melhor modelo
model = ConvNet() # instancia o modelo
optimizer = Adam(model.parameters(), lr=learning_rate) # instancia o otimizador
checkpoint = torch.load('best_model.pth') # carrega o checkpoint
model.load_state_dict(checkpoint['model_state_dict']) # carrega os pesos
epochs = checkpoint['epoch'] 


